In [9]:
import os

import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import sys

import matplotlib.pylab as pl

# set data path
dir_path = pathlib.Path().absolute()            # 현재 이 노트북 파일 위치
file = '../data/MOM4_data.csv'                  # 현재 이 노트북 파일 기준 디렉토리 위치에서 데이터셋의 위치
data_path = os.path.join(dir_path, file)
print('MOM4 data is located at:',data_path)

MOM4 data is located at: /media/sda/Documents/github/angle_analysis/../data/MOM4_data.csv


In [10]:
''' Read dataset '''
# configurations for plots
%matplotlib inline
%time # for timing
pd.set_option('display.max_columns', None)

# read data
df = pd.read_csv(data_path  ,index_col=False).drop(['Unnamed: 0'], axis=1)
df.reset_index(drop=True, inplace=True)
assert df.isnull().sum().sum() == 0
df.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


,CRD,PartType,Orient.,X,Y,SPI_VOLUME1,SPI_VOLUME2,SPI_X1,SPI_Y1,SPI_X2,SPI_Y2,SPI_L1,SPI_W1,SPI_L2,SPI_W2,SPI_L,SPI_W,PRE_X,PRE_Y,PRE_A,PRE_L,PRE_W,POST_X,POST_Y,POST_A,POST_L,POST_W,Job,Chip_L,Chip_W,SPI_R,PRE_R,POST_R,SPI_VOLUME_MEAN,SPI_VOLUME_DIFF
0,B1,R0402,90,31.26,100.11,81.7355,75.6950,-51.9,63.3,-52.1,60.5,63.3,51.9,60.5,52.1,61.90,52.0,-19.887293,-1.840963,-1.877,-1.840963,19.887293,-7.440417,28.108231,1.613,28.108231,7.440417,CENTER,400.0,200.0,80.843120,19.972320,29.076321,78.71525,6.0405
1,B2,R0402,90,31.83,100.11,83.1555,71.5083,-45.7,66.1,-48.3,61.3,66.1,45.7,61.3,48.3,63.70,47.0,-26.766272,6.898049,-1.449,6.898049,26.766272,5.487133,22.242701,0.000,22.242701,-5.487133,CENTER,400.0,200.0,79.162428,27.640847,22.909526,77.33190,11.6472
2,B3,R0402,90,32.40,100.11,87.0870,67.2606,-48.6,75.8,-34.6,68.9,75.8,48.6,68.9,34.6,72.35,41.6,-32.058273,8.125244,-3.278,8.125244,32.058273,-19.644203,12.971593,1.552,12.971593,19.644203,CENTER,400.0,200.0,83.457070,33.071929,23.540538,77.17380,19.8264
3,B4,R0402,90,32.97,100.11,80.0433,75.8866,-56.3,65.8,-44.9,60.4,65.8,56.3,60.4,44.9,63.10,50.6,-23.437572,-6.195457,-3.337,-6.195457,23.437572,-22.517658,26.396389,0.000,26.396389,22.517658,CENTER,400.0,200.0,80.882446,24.242596,34.696027,77.96495,4.1567
4,B5,R0402,90,33.54,100.11,76.7088,74.0452,-48.9,69.7,-51.1,69.2,69.7,48.9,69.2,51.1,69.45,50.0,-5.187286,7.085744,-3.408,7.085744,5.187286,-21.413681,29.193306,3.611,29.193306,21.413681,CENTER,400.0,200.0,85.576296,8.781555,36.204901,75.37700,2.6636


In [11]:
# '''
# Analysis: determine which features (e.g. PRE X,Y / SPI X,Y / VOLUME, etc) to feed in to the model
# - post angle v
# from numpy.lib.shape_base import splits. distance(spi,pad기준) 순서대로 나열했을 때 그룹별 mean, std, 그리고 std을 따르는 curve fitting with uncertainty(Poly, exponential)
# - SPI/PAD 기준 distance offset vs. spi volume
# - volume, offset distance 꽈 angle 의 직접적인 관계? ....
# (volume -> higher post distance and higher objective value? distance -> higher post angle?)
# '''

# # Comparison 1: 
# #   Observe Post-AOI angle vs. offset distance from SPI & Pad positions

# # Comparison 2: 
# #   Observe Post-AOI angle vs. offset distance from SPI & Pad positions
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import PolynomialFeatures
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
# from sklearn.pipeline import make_pipeline
# import sys

# import matplotlib.pylab as pl
# colors = pl.cm.jet(np.linspace(0,1,3))

# # switch 90 data to 0 data
# def switchOrient(x90, y90):
#     y0 = float(x90)
#     x0 = float(-y90)
#     return x0, y0 

# abs_post_angle = True                                               # abs(post_angle) boolean flag
# dist_in_chip_length_percentage = False
# dfgroups = df.groupby(['PartType'])                                 # group by chip type
# dists = {}
# for idx, (name, group) in enumerate(dfgroups):                      # iterate through each grouped data
#     dists[f'{name}'] = []
    
#     group = group.drop(columns=['CRD','PartType','Job'],axis=1).astype(float)
#     pad1_x, pad1_y, pad2_x, pad2_y = 0, 0, 0, 0
#     pad_minus_chip_dist = []
#     spi_minus_chip_dist = []
    
#     post_a = group['POST_A']
#     chip_length = group['Chip_L']
    
#     if abs_post_angle == True:
#         post_a = np.abs(post_a)
    
#     spi_v = []
#     for index, row in group.iterrows():                             # iterate through each row in each data group
#         spi_x1, spi_y1, spi_x2, spi_y2, post_x, post_y, x, y, chip_l, chip_w, orient, spi_v1, spi_v2 = \
#             row[['SPI_X1', 'SPI_Y1', 'SPI_X2', 'SPI_Y2', 'POST_X', 'POST_Y', 'X', 'Y', 'Chip_L','Chip_W', 'Orient.','SPI_VOLUME1','SPI_VOLUME2']].astype(float)
#         x, y = float(x), float(y)
        
#         spi_v.append((spi_v1 + spi_v2) / 2)
        
#         # switch 90 degree x, y to 0 degree x, y space
#         if orient == '90':
#             x, y = switchOrient(x,y)
#             spi_x1, spi_y1 = switchOrient(spi_x1, spi_y1)
#             spi_x2, spi_y2 = switchOrient(spi_x2, spi_y2)
#             post_x, post_y = switchOrient(post_x, post_y)
#         # compute pad locations (pad is separated apart by (chip_length/2) in both 0 and 90 degree cases)
#         pad1_x = x - (chip_l // 2)
#         pad1_y = y
#         pad2_x = x + (chip_l // 2)
#         pad2_y = y
#         # compute mean pad (for distance simplicity)
#         pad_x = (pad1_x + pad2_x) / 2
#         pad_y = (pad1_y + pad2_y) / 2
#         # compute distance 
#         x = np.abs(pad_x - post_x)
#         y = np.abs(pad_y - post_y)
#         pad_minus_chip_dist.append(np.linalg.norm((x,y)))

#         # compute mean SPI (for distance simplicity)
#         spi_x = (spi_x1 + spi_x2) / 2
#         spi_y = (spi_y1 + spi_y2) / 2
#         # compute distance
#         x = np.abs(spi_x - post_x)
#         y = np.abs(spi_y - post_y)
#         spi_minus_chip_dist.append(np.linalg.norm((x, y)))
    
#     # sort (distance, post_angle) by (distance) and stack all columns for each group
#     pad_chip_postangle_data = np.column_stack((pad_minus_chip_dist, post_a, spi_v))
#     pad_chip_postangle_data = pad_chip_postangle_data[pad_chip_postangle_data[:,0].argsort()] # sort by distance
#     spi_chip_postangle_data = np.column_stack((spi_minus_chip_dist, post_a, spi_v))
#     spi_chip_postangle_data = spi_chip_postangle_data[spi_chip_postangle_data[:,0].argsort()] # sort by distance
#     print(f'[COUNT] Chip: {name} \
#             \n PAD (distance/angle): {len(pad_chip_postangle_data)}, {len(post_a)} \
#             \n SPI (distance/angle): {len(spi_chip_postangle_data)}, {len(post_a)}')
    
#     # split by distance groups
#     # for 60750 points, divide into 15 groups by index (e.g. 4050 samples each)
#     num_groups = 15
#     num_samples = len(post_a) // num_groups # 4050 samples
    
#     ''' split the distances into 15 groups for computing the statistics: Post Angle (mean, std) '''
#                                                                         # x
#     pad_means, pad_stds, spi_means, spi_stds = [], [], [], []           # y
#     for split_idx in np.arange(start=0, stop=num_groups, step=1):
#         # mean and std of post angle
#         pad_mean = np.mean(pad_chip_postangle_data[split_idx:(split_idx+1)*num_samples, 1])
#         pad_std = np.std(pad_chip_postangle_data[split_idx:(split_idx+1)*num_samples, 1])
#         spi_mean = np.mean(spi_chip_postangle_data[split_idx:(split_idx+1)*num_samples, 1])
#         spi_std = np.std(spi_chip_postangle_data[split_idx:(split_idx+1)*num_samples, 1])
        
#         # median dist
#         temp = pd.DataFrame(pad_chip_postangle_data).copy().apply(np.floor).astype(np.int).loc[:,0].values.tolist()
#         to_append = np.mean(temp[split_idx:(split_idx+1)*num_samples])
#         if dist_in_chip_length_percentage == True:
#             to_append = to_append / chip_length.values[0] * 100 # percentage of chip length
#         dists[f'{name}'].append(to_append)        
        
#         pad_means.append(pad_mean)
#         pad_stds.append(pad_std)
#         spi_means.append(spi_mean)
#         spi_stds.append(spi_std)
#     # print(f'chip: {name}, {pad_means}, {pad_stds}, {spi_means}, {spi_stds}')
#     if dist_in_chip_length_percentage == True:
#         pad_minus_chip_dist = pad_minus_chip_dist / chip_length.values[0] * 100
#         spi_minus_chip_dist = spi_minus_chip_dist / chip_length.values[0] * 100
    
#     # polynomial curve fitting
#     degree = 3
#     coeff = np.polyfit(dists[f'{name}'], pad_stds, degree)
#     poly = np.poly1d(coeff)
#     poly_y_pad = poly(dists[f'{name}'])
    
#     coeff = np.polyfit(dists[f'{name}'], spi_stds, degree)
#     poly = np.poly1d(coeff)
#     poly_y_spi = poly(dists[f'{name}'])
    
#     ''' plot mean and std  '''
#     # x: spi-chip or pad-chip distance (um), y: post angle stats (degree)
#     fig = plt.figure(figsize=(10,10))
    
#     ax = fig.add_subplot(221)
#     ax.plot(dists[f'{name}'], pad_means, 'r.', label=f'{name}: mean')
#     ax.plot(dists[f'{name}'], pad_stds, 'b.', label=f'{name}: std dev')
#     ax.plot(dists[f'{name}'], poly_y_pad, marker='', color='green', label=f'{name}: std dev curve fit (deg={degree})')
#     lower = [(m-s) for m,s in zip(pad_means, pad_stds)]
#     upper = [(m+s) for m,s in zip(pad_means, pad_stds)]
#     ax.fill_between(dists[f'{name}'], lower, upper, where=upper>lower, facecolor='green', alpha=0.2)
#     ax.set_title(f'{name}: Post Angle vs. Pad-Chip offset distance')
#     if dist_in_chip_length_percentage:
#         ax.set_xlabel('Pad-Chip vector offset distance in percentage of chip length (%)')
#     else:
#         ax.set_xlabel('Pad-Chip vector offset distance (\u03BCm)')
#     ax.set_ylabel('Post Angle (degree)')
#     ax.set_xlim([np.min(dists[f'{name}']), np.max(dists[f'{name}'])])
#     ax.set_ylim([-2, 6])
#     ax.legend()
    
#     ax2 = fig.add_subplot(222)
#     ax2.plot(dists[f'{name}'], spi_means, 'r.', label=f'{name}: mean')
#     ax2.plot(dists[f'{name}'], spi_stds, 'b.', label=f'{name}: std dev')
#     ax2.plot(dists[f'{name}'], poly_y_spi, marker='', color='green', label=f'{name}: std dev curve fit (deg={degree})')
#     lower = [(m-s) for m,s in zip(spi_means, spi_stds)]
#     upper = [(m+s) for m,s in zip(spi_means, spi_stds)]
#     ax2.fill_between(dists[f'{name}'], lower, upper, where=upper>lower, facecolor='green', alpha=0.2)
#     ax2.set_title(f'{name}: Post Angle vs. SPI-Chip offset distance')
#     if dist_in_chip_length_percentage:
#         ax2.set_xlabel('SPI-Chip vector offset distance in percentage of chip length (%)')
#     else:
#         ax2.set_xlabel('SPI-Chip vector offset distance (\u03BCm)')
#     ax2.set_ylabel('Post Angle (degree)')
#     ax2.set_xlim([np.min(dists[f'{name}']), np.max(dists[f'{name}'])])
#     ax2.set_ylim([-2, 6])
#     ax2.legend()
    
#     ax3 = fig.add_subplot(223)
#     ax3.plot(pad_minus_chip_dist, spi_v, 'r.', label=f'{name}')
#     ax3.set_title(f'{name}: SPI Volume vs. Pad-Chip offset distance')
#     if dist_in_chip_length_percentage:
#         ax3.set_xlabel('Pad-Chip vector offset distance in percentage of chip length (%)')
#     else:
#         ax3.set_xlabel('Pad-Chip vector offset distance (\u03BCm)')
#     ax3.set_ylabel('SPI Volume (%)')
#     # ax3.set_xlim([np.min(pad_minus_chip_dist), np.max(pad_minus_chip_dist)])
#     ax3.set_xlim([np.min(dists[f'{name}']), np.max(dists[f'{name}'])])
#     ax3.set_ylim([np.min(spi_v), np.max(spi_v)])
#     ax3.legend()      
    
#     ax4 = fig.add_subplot(224)
#     ax4.plot(spi_minus_chip_dist, spi_v, 'r.', label=f'{name}')
#     ax4.set_title(f'{name}: SPI Volume vs. SPI-Chip offset distance')
#     if dist_in_chip_length_percentage:
#         ax4.set_xlabel('SPI-Chip vector offset distance in percentage of chip length (%)')
#     else:
#         ax4.set_xlabel('SPI-Chip vector offset distance (\u03BCm)')
#     ax4.set_ylabel('SPI Volume (%)')
#     # ax4.set_xlim([np.min(pad_minus_chip_dist), np.max(pad_minus_chip_dist)])
#     ax4.set_xlim([np.min(dists[f'{name}']), np.max(dists[f'{name}'])])
#     ax4.set_ylim([np.min(spi_v), np.max(spi_v)])
#     ax4.legend()        
    
#     fig.tight_layout()
#     if abs_post_angle == True:  
#         fig.savefig(f'{name}_distance_postangle_abs.png')
#     else:
#         fig.savefig(f'{name}_distance_postangle.png')
#     plt.show()
    
#     # plot angle vs. spi volume during a given offset distance
#     pad_chip_postangle_data = pd.DataFrame(data=pad_chip_postangle_data, columns=[f'dist_{name}', 'post_angle', 'spi_volume'])
#     spi_chip_postangle_data = pd.DataFrame(data=spi_chip_postangle_data, columns=[f'dist_{name}', 'post_angle', 'spi_volume'])

#     fig2 = plt.figure(figsize=(10,6))
#     ax1 = fig2.add_subplot(121)
#     xdata = 'spi_volume'
#     ydata = 'post_angle'
#     pad_chip_postangle_data.plot(ax = ax1, y=ydata, x=xdata, style='o')
#     ax2 = fig2.add_subplot(122)
#     spi_chip_postangle_data.plot(ax = ax2, y=ydata, x=xdata, style='o')

#     ax1.set_title(f'{name}: Post-AOI angle vs. SPI Volume (from Pad)')
#     ax1.set_xlabel(f'{xdata}')
#     ax1.set_ylabel(f'{ydata}')
#     ax1.legend()        

#     ax2.set_title(f'{name}: Post-AOI angle vs. SPI Volume (from SPI)')
#     ax2.set_xlabel(f'{xdata}')
#     ax2.set_ylabel(f'{ydata}')
#     ax2.legend()        

#     fig2.tight_layout()
#     fig2.savefig(f'{name}_angle_volume.png')
    
    
# plt.close()
    
    

In [12]:
colors = pl.cm.jet(np.linspace(0,1,3))

# switch 90 data to 0 data
def switchOrient(x90, y90):
    y0 = float(x90)
    x0 = float(-y90)
    return x0, y0 

abs_post_angle = True                                               # abs(post_angle) boolean flag
dist_in_chip_length_percentage = False
dfgroups = df.groupby(['PartType'])                                 # group by chip type
dists = {}
for idx, (name, group) in enumerate(dfgroups):                      # iterate through each grouped data
    dists[f'{name}'] = []
    
    group = group.drop(columns=['CRD','PartType','Job'],axis=1).astype(float)
    pad1_x, pad1_y, pad2_x, pad2_y = 0, 0, 0, 0
    pad_minus_chip_dist = []
    spi_minus_chip_dist = []
    
    post_a = group['POST_A']
    chip_length = group['Chip_L']
    
    if abs_post_angle == True:
        post_a = np.abs(post_a)
    
    spi_v = []
    for index, row in group.iterrows():                             # iterate through each row in each data group
        spi_x1, spi_y1, spi_x2, spi_y2, post_x, post_y, x, y, chip_l, chip_w, orient, spi_v1, spi_v2 = \
            row[['SPI_X1', 'SPI_Y1', 'SPI_X2', 'SPI_Y2', 'POST_X', 'POST_Y', 'X', 'Y', 'Chip_L','Chip_W', 'Orient.','SPI_VOLUME1','SPI_VOLUME2']].astype(float)
        x, y = float(x), float(y)
        
        spi_v.append((spi_v1 + spi_v2) / 2)
        
        # switch 90 degree x, y to 0 degree x, y space
        if orient == '90':
            x, y = switchOrient(x,y)
            spi_x1, spi_y1 = switchOrient(spi_x1, spi_y1)
            spi_x2, spi_y2 = switchOrient(spi_x2, spi_y2)
            post_x, post_y = switchOrient(post_x, post_y)
        # compute pad locations (pad is separated apart by (chip_length/2) in both 0 and 90 degree cases)
        pad1_x = x - (chip_l // 2)
        pad1_y = y
        pad2_x = x + (chip_l // 2)
        pad2_y = y
        # compute mean pad (for distance simplicity)
        pad_x = (pad1_x + pad2_x) / 2
        pad_y = (pad1_y + pad2_y) / 2
        # compute distance 
        x = np.abs(pad_x - post_x)
        y = np.abs(pad_y - post_y)
        pad_minus_chip_dist.append(np.linalg.norm((x,y)))

        # compute mean SPI (for distance simplicity)
        spi_x = (spi_x1 + spi_x2) / 2
        spi_y = (spi_y1 + spi_y2) / 2
        # compute distance
        x = np.abs(spi_x - post_x)
        y = np.abs(spi_y - post_y)
        spi_minus_chip_dist.append(np.linalg.norm((x, y)))
    
    # sort (distance, post_angle) by (distance) and stack all columns for each group
    pad_data = np.column_stack((pad_minus_chip_dist, post_a, spi_v))
    pad_data = pad_data[pad_data[:,0].argsort()] # sort by distance
    pad_data = pd.DataFrame(pad_data, columns=['pad_dist','post_angle','spi_volume'])
    
    spi_data = np.column_stack((spi_minus_chip_dist, post_a, spi_v))
    spi_data = spi_data[spi_data[:,0].argsort()] # sort by distance
    spi_data = pd.DataFrame(spi_data, columns=['spi_dist','post_angle','spi_volume'])
    
    print(f'[COUNT] Chip: {name} \
            \n Pad (distance/angle): {len(pad_data)}, {len(post_a)} \
            \n SPI (distance/angle): {len(spi_data)}, {len(post_a)}')
    
    num_groups = 15
    pad_range = pad_data.iloc[:,0].max() - pad_data.iloc[:,0].min()
    pad_max_dists = [i * pad_range // num_groups for i in np.arange(num_groups)]
    
    spi_range = spi_data.iloc[:,0].max() - spi_data.iloc[:,0].min()
    spi_max_dists = [i * spi_range // num_groups for i in np.arange(num_groups)]
    
    print('[max dists]\npad:', pad_max_dists, '\nspi:', spi_max_dists)
    
    pad_plot_data = []
    spi_plot_data = []
    for i, (pad_max, spi_max) in enumerate(zip(pad_max_dists[1:], spi_max_dists[1:])):
        pad = pad_data.iloc[:,0]
        # print(pad_max_dists[i], '----', pad_max_dists[i+1])
        pad_valid_df = pad[(pad < pad_max_dists[i+1]) & (pad > pad_max_dists[i])]
        pad_valid_idx = pad_valid_df.index
        if len(pad_valid_idx) == 0:
            continue
        
        spi = spi_data.iloc[:,0]
        # print(spi_max_dists[i], '----', spi_max_dists[i+1])
        spi_valid_df = spi[(spi < spi_max_dists[i+1]) & (spi > spi_max_dists[i])]
        spi_valid_idx = spi_valid_df.index
        
        # fetch other data from pad_data or spi_data using index pad_valid_idx, spi_valid_idx
        # columns: distance, post angle, spi volume
        pad_group_df = pad_data.iloc[pad_valid_idx, :]
        spi_group_df = spi_data.iloc[spi_valid_idx, :]
        # print('[group df]\n pad:',pad_group_df, '\nspi:',spi_group_df)
        
        # compute statistics for the angle
        pad_mean = np.mean(pad_group_df.iloc[:, 2])
        pad_std = np.std(pad_group_df.iloc[:, 2])
        
        spi_mean = np.mean(spi_group_df.iloc[:, 2])
        spi_std = np.std(spi_group_df.iloc[:, 2])
        
        # select at which distance to plot this mean and std
        # > (1) mean (2) median (3) mean of the min and max values
        # pad_dist_avg = np.median(pad_group_df.iloc[:,0])
        # spi_dist_avg = np.median(spi_group_df.iloc[:,0])
        pad_dist_avg = np.mean([pad_group_df.iloc[:,0].max(), pad_group_df.iloc[:,0].min()])
        spi_dist_avg = np.mean([spi_group_df.iloc[:,0].max(), spi_group_df.iloc[:,0].min()])
        
        if not np.isnan(pad_dist_avg):
            pad_plot_data = np.concatenate((pad_plot_data, np.column_stack((pad_dist_avg, pad_mean, pad_std))[0]), axis=0)
        if not np.isnan(spi_dist_avg):
            spi_plot_data = np.concatenate((spi_plot_data, np.column_stack((spi_dist_avg, spi_mean, spi_std))[0]), axis=0)
        print(pad_plot_data, spi_plot_data)
    
        
    # if empty (not less than the max dist, it is shown as 'NaN')
    print('[plot data]\n pad:', pad_plot_data, '\n spi:', spi_plot_data)
    
    

[COUNT] Chip: R0402             
 Pad (distance/angle): 60750, 60750             
 SPI (distance/angle): 60750, 60750
[max dists]
pad: [0.0, 19.0, 39.0, 59.0, 79.0, 99.0, 119.0, 139.0, 159.0, 179.0, 198.0, 218.0, 238.0, 258.0, 278.0] 
spi: [0.0, 18.0, 36.0, 54.0, 72.0, 90.0, 109.0, 127.0, 145.0, 163.0, 181.0, 200.0, 218.0, 236.0, 254.0]
[35.3726447  79.93992222  6.4618169 ] [27.00088477 83.89735928  6.58050833]
[35.3726447  79.93992222  6.4618169  49.99749845 75.69124271  5.14589901] [27.00088477 83.89735928  6.58050833 45.00015371 80.72748193  6.9699366 ]
[35.3726447  79.93992222  6.4618169  49.99749845 75.69124271  5.14589901
 69.00222238 76.39146676  6.7784803 ] [27.00088477 83.89735928  6.58050833 45.00015371 80.72748193  6.9699366
 62.99938418 77.05736068  6.9841959 ]
[35.3726447  79.93992222  6.4618169  49.99749845 75.69124271  5.14589901
 69.00222238 76.39146676  6.7784803  89.00053929 79.47906302  7.15485086] [27.00088477 83.89735928  6.58050833 45.00015371 80.72748193  6.96993